This notebook is associated with the paper "The relative class number one problem for function fields, III" by K.S. Kedlaya. It runs in SageMath (tested using version 9.5rc3) and depends on Magma (tested using version
2.26-9).

In this notebook, we identify generic curves of genus 6 which are candidates for the curve $C$ in a purely geometric quadratic extension $F'/F$ of relative class number 1 under the assumption that either $\#C(\mathbb{F}_2) > 4$, or $\#C(\mathbb{F}_2) = 4$ and the $\mathbb{F}_2$-points do not form a $g^1_4$.  Allow 3 hours for completion.

In [1]:
load("preamble.sage")

Construct the set of $\mathbb{F}_2$-rational points of $G(2,5)$, represented as $2 \times 5$ echelon matrices,
together with their Plucker coordinates.

In [2]:
F = GF(2)

In [3]:
coords = {}
for V in VectorSpace(F, 5).subspaces(2):
    M = V.matrix()
    M.set_immutable()
    coords[M] = vector(M.minors(2))
S = list(coords.keys())
len(S)

155

Construct the Plucker relations defining $G(2,5)$ inside $\mathbb{P}^9$.

In [4]:
P.<x01,x02,x03,x04,x12,x13,x14,x23,x24,x34> = PolynomialRing(F, 10)
quads = (x01*x23 + x02*x13 + x03*x12,
         x01*x24 + x02*x14 + x04*x12,
         x01*x34 + x03*x14 + x04*x13,
         x02*x34 + x03*x24 + x04*x23,
         x12*x34 + x13*x24 + x14*x23)

Reat representative tuples of points on $G(2,5)$ from a file.

In [5]:
with open("genus6-tuples.txt", "r") as f:
    s = f.read()
    l = sage_eval(s)
    l = [[vector(F, v) for v in vecs] for vecs in l]
len(l)

2475

Construct the linear spans of the tuples of length greater than 4.

In [6]:
d = {}
for vecs in l:
    if len(vecs) == 4:
        m = Matrix(vecs).echelon_form()
        W = m.row_space()
        for v in vecs:
            v.set_immutable()
        d[tuple(vecs)] = W

In [7]:
[(i, sum(1 for vecs in d if d[vecs].dimension() == i)) for i in range(7)]

[(0, 0), (1, 0), (2, 0), (3, 7), (4, 32), (5, 0), (6, 0)]

Construct 6-dimensional extensions of these subspaces (in terms of bases thereof).

In [8]:
def vecs_to_gens(vecs):
    return tuple(sum(P.gens()[i] * v[i] for i in range(10)) for v in vecs)

In [9]:
def linear_section(vecs):
    V = Matrix(vecs).right_kernel()
    tmp2 = vecs_to_gens(V.gens()) + quads
    return P.ideal(tmp2)

In [10]:
d2 = defaultdict(list)
V0 = VectorSpace(F, 10)
for vecs in d:
    V1 = d[vecs]
    if V1.dimension() <= 3:
        continue
    for V2 in subspaces_containing(V0, V1, 6-V1.dimension()):
        J = linear_section(V2.gens())
        if J.dimension() > 3:
            continue
        d2[vecs].append(V2)

In [11]:
[sum(len(d2[vecs]) for vecs in d2 if len(vecs) == i) for i in range(4,7)]

[18982, 0, 0]

For each 6-dimensional subspace, identify quadrics which meet this subspace in exactly the specified set of $\mathbb{F}_2$-rational points, modulo multiples of the linear relations cutting out the subspaces and the quadrics cutting out the Grassmannian.

In [12]:
monos2 = [prod(x) for x in itertools.combinations_with_replacement(P.gens(), 2)]
len(monos2)

55

In [13]:
coords2 = {}
for x in V0:
    x.set_immutable()
    coords2[x] = vector(F, (mu(*x) for mu in monos2))

In [14]:
def redundancy(gens, F=F, P=P, monos2=monos2, quads=quads):
    return [vector(F, ((gen*y).coefficient(mu) for mu in monos2)) for gen in gens for y in P.gens()] + \
       [vector(F, (gen.coefficient(mu) for mu in monos2)) for gen in quads]

In [ ]:
curves = defaultdict(list)
for vecs in d2:
    s = len(vecs)
    for V in d2[vecs]:
        V1 = V.matrix().right_kernel()
        gens = vecs_to_gens(V1.basis())
        pts = []
        for x in V:
            if x != 0:
                x.set_immutable()
                if all(gen(*x) == 0 for gen in quads):
                    pts.append(x)
        perp = Matrix([coords2[x] for x in pts])
        target = vector(F, (0 if x in vecs else 1 for x in pts))
        for w in solve_right_iterator(perp, target, redundancy, gens):
            gens1 = sum(w[i]*monos2[i] for i in range(55))
            curves[(s,)].append(gens + (gens1,))

In [ ]:
[(s, len(curves[s])) for s in curves]

Enforce the desired point counts over $\mathbb{F}_{2^i}$ for $i=2$ using point counting and for $i=3,4$ using commutative algebra.

In [ ]:
F4 = GF(4)
S4 = []
for V in VectorSpace(F4, 5).subspaces(2):
    M = V.matrix()
    v = vector(M.minors(2))
    i = min(j for j in range(10) if v[j])
    assert v[i] == 1
    v.set_immutable()
    S4.append(v)
len(S4)

In [ ]:
tmp = [s for s in curves if len(s) == 1]
tmp2 = set(t[:2] for t in targets6)
for (s1,) in tmp:
    gens1 = 0
    for gens in curves[(s1,)]:
        if gens1 != gens[:-1]:
            gens1 = gens[:-1]
            pts2 = [x for x in S4 if all(gen(*x) == 0 for gen in gens1)]
        s2 = sum(1 for x in pts2 if gens[-1](*x) == 0)
        if (s1, s2) in tmp2:
            curves[(s1,s2)].append(gens)
    del curves[(s1,)]

In [ ]:
[(s, len(curves[s])) for s in curves]

In [ ]:
def count_by_ideal(gens, n):
    J = P.ideal(gens + quads + tuple(y^(2^n) + y for y in P.gens()))
    return (J.vector_space_dimension() - 1) // (2^n-1)

In [ ]:
for n in range(3, 5):
    tmp = set(t[:n] for t in targets6)
    tmp2 = list(curves.keys())    
    for s in tmp2:
        for gens in curves[s]:
            i = count_by_ideal(gens, n)
            s1 = s + (i,)
            if s1 in tmp:
                curves[s1].append(gens)
        del curves[s]
    print([(s, len(curves[s])) for s in curves])

Close out this case.

In [ ]:
proj = magma.ProjectiveSpace(P)
curves = {s: [gens + quads for gens in curves[s]] for s in curves}
closeout(curves, X=proj, genus=6)